# Gas Station Optimization Problem
#### _Edward Krueger, edkrueger@gmail.com_

## Purpose

Learn how to modularize a simulation so that we can seach over different inputs to find an optimum.

## Overview

Now that you have completed Alice's simulation, she want you to answer her over all question: How many pumps should she have at her gas station. She wants to know how many pumps are needed to maximize gallons sold.  In order to do this we will impove on the code from the Gas Station Simulation Problem. Feel free to use either your own code or the code in the solution as a starting point. 

## The details

### Encapsulate the Simulation

We need to rewrite the simulation as a encapsulated function that takes every varible the simulation needs to run as an argument. This will require that for every function or object, the parameters are passed explicitly as arguments; rather than being read as global varialbes.

This will allow us to easily see the effect of gas pumps on the number of gallons sold.

Tips:
* We can either eliminate the print statement of include a flag telling whether or not the simulation should print results. The simulation and optimization will run much faster if they don't have to print and the output will be much cleaner.
* Since we are only focused on the number of gallons sold, we don't need the other metrics. Since the GasStation instance keeps track of all of the gas sold, we don't need to save the Car instances in a list.

### Simulate for Different Numbers of Pumps

Now since we have a function that returns the simulated number of gallons sold, we can to run this function for a number of values to see its results.

# Gas Station Optimization Solution

## Setup

In [1]:
import simpy

import numpy as np
import pandas as pd

from scipy.stats import expon
from scipy.stats import norm

In [2]:
class GasStation():
    
    """
    Models a gas station as a simpy.Resource with a given number of pumps.
    
    Arguments:
    -----------
    num_pumps (int): The number of pumps at the gas station
    env (simpy.Environment): The simpy environment
    """
    
    def __init__(self, num_pumps, env):
        
        self.num_pumps = num_pumps
        
        self.env = env
        
        self.resource = simpy.Resource(self.env, capacity=num_pumps)
        
        self.minutes_utilized = 0
        self.gallons_sold = 0

In [3]:
class Car():

    """
    Models a car.
    The car's simulation process in the simulation is the method .run().
    
    Arguments:
    -----------
    id (int): A unique id for the car
    gas_required (float): Amount of gallons a car will fill
    lay_time (float): Extra minutes that the car stays at the pump
    gas_station (GasStation): The gas station the cars are going to
    env (simpy.Environment): The simpy environment
    
    """
    
    def __init__(
        self,
        id,
        gas_required,
        lay_time,
        gas_station,
        pump_rate,
        env,
        verbose
    ):
        
        self.id = id
        self.gas_required = gas_required
        self.lay_time = lay_time
        self.gas_station = gas_station
        self.pump_rate = pump_rate
        self.env = env
        
        self.verbose = verbose
        
        self.wait_time = np.inf
        self.finished = False
        
        self.action = self.env.process(self.run())
        
    def run(self):
        
        queue_time = self.env.now
        
        if self.verbose:
            print(f'Car {self.id} arrives at {round(queue_time, 2)} minutes')
                
        with self.gas_station.resource.request() as req:
            
            yield req
            
            pump_start_time = self.env.now
            
            if self.verbose:
                print(f'Car {self.id} begins utilizing a pump at {round(pump_start_time, 2)} minutes')

            pump_time = self.gas_required / self.pump_rate
            utilization = max(pump_time, self.lay_time)
            
            yield self.env.timeout(utilization)
            
            pump_end_time = self.env.now
            
            if self.verbose:
                print(f'Car {self.id} leaves its pump at {round(pump_end_time, 2)} minutes')
            
            self.finished = True
            
            self.wait_time = pump_start_time - queue_time
            self.gas_station.minutes_utilized += utilization
            self.gas_station.gallons_sold += self.gas_required

In [4]:
def scheduler(
    expected_wait,
    gas_required_mean,
    gas_required_std,
    pump_rate,
    expected_lay_time,
    gas_station,
    cars,
    env,
    verbose
):
    
    """
    Controls the arrival of cars at the gas station.
    
    Arguments:
    -----------
    env (simpy.Environment): The simpy environment
    """
    
    id = 0
    
    while True:
        
        waiting_time = expon.rvs(loc=0, scale=expected_wait)
        
        std_norm = norm.rvs()
        gas_required = std_norm * gas_required_std + gas_required_mean
        gas_required = max([0, gas_required])
        
        lay_time = expon.rvs(loc=0, scale=expected_lay_time)
        
        yield env.timeout(waiting_time)
        
        cars.append(
            Car(
                id=id,
                gas_required=gas_required,
                lay_time=lay_time,
                gas_station=gas_station,
                pump_rate = pump_rate,
                env=env,
                verbose=verbose
            )
        )
        
        id += 1

In [5]:
def sim(
    num_pumps,
    expected_wait,
    gas_required_mean,
    gas_required_std,
    pump_rate,
    expected_lay_time,
    sim_time,
    verbose = False
):
    
    env = simpy.Environment()
    cars = []
    gas_station = GasStation(num_pumps, env)
    env.process(
        scheduler(
            expected_wait,
            gas_required_mean,
            gas_required_std,
            pump_rate,
            expected_lay_time,
            gas_station,
            cars,
            env,
            verbose=verbose
        )
    )
    env.run(until=sim_time)
    
    total_gallons_sold = gas_station.gallons_sold
    
    return total_gallons_sold

## Create Fixed Parameters

In [6]:
fixed_parameters = {
    'expected_wait': 2,
    'gas_required_mean': 15,
    'gas_required_std': 5,
    'pump_rate': 10,
    'expected_lay_time': 5,
    'sim_time': 24 * 60,
}

## Do a test run of the simulation

In [7]:
gas_sold = sim(
    num_pumps=1,
    **fixed_parameters,
    verbose=True
)

gas_sold

Car 0 arrives at 2.97 minutes
Car 0 begins utilizing a pump at 2.97 minutes
Car 1 arrives at 3.36 minutes
Car 0 leaves its pump at 4.58 minutes
Car 1 begins utilizing a pump at 4.58 minutes
Car 2 arrives at 9.58 minutes
Car 1 leaves its pump at 9.58 minutes
Car 2 begins utilizing a pump at 9.58 minutes
Car 3 arrives at 11.83 minutes
Car 4 arrives at 13.57 minutes
Car 5 arrives at 13.76 minutes
Car 6 arrives at 14.6 minutes
Car 7 arrives at 15.67 minutes
Car 8 arrives at 18.74 minutes
Car 9 arrives at 23.69 minutes
Car 10 arrives at 27.09 minutes
Car 11 arrives at 29.3 minutes
Car 12 arrives at 29.93 minutes
Car 13 arrives at 30.11 minutes
Car 14 arrives at 30.43 minutes
Car 15 arrives at 30.62 minutes
Car 16 arrives at 30.65 minutes
Car 17 arrives at 30.81 minutes
Car 18 arrives at 31.44 minutes
Car 2 leaves its pump at 31.54 minutes
Car 3 begins utilizing a pump at 31.54 minutes
Car 19 arrives at 31.65 minutes
Car 20 arrives at 31.98 minutes
Car 21 arrives at 32.19 minutes
Car 22 arri

Car 557 arrives at 1144.25 minutes
Car 558 arrives at 1144.44 minutes
Car 559 arrives at 1147.95 minutes
Car 218 leaves its pump at 1148.45 minutes
Car 219 begins utilizing a pump at 1148.45 minutes
Car 219 leaves its pump at 1150.84 minutes
Car 220 begins utilizing a pump at 1150.84 minutes
Car 560 arrives at 1152.09 minutes
Car 220 leaves its pump at 1152.53 minutes
Car 221 begins utilizing a pump at 1152.53 minutes
Car 221 leaves its pump at 1153.98 minutes
Car 222 begins utilizing a pump at 1153.98 minutes
Car 561 arrives at 1155.11 minutes
Car 562 arrives at 1156.32 minutes
Car 563 arrives at 1156.77 minutes
Car 222 leaves its pump at 1159.25 minutes
Car 223 begins utilizing a pump at 1159.25 minutes
Car 223 leaves its pump at 1160.41 minutes
Car 224 begins utilizing a pump at 1160.41 minutes
Car 564 arrives at 1161.09 minutes
Car 565 arrives at 1161.64 minutes
Car 224 leaves its pump at 1162.14 minutes
Car 225 begins utilizing a pump at 1162.14 minutes
Car 225 leaves its pump at 

4096.227543897437

## Do the optimization

In [8]:
opt_table = pd.Series(
    {num_pumps: sim(num_pumps, **fixed_parameters) for num_pumps in range(1,100)}
)

In [9]:
arg_max = opt_table.idxmax()
max_gallons_sold = opt_table[arg_max]

print(f'The maximum number of gallons sold is {max_gallons_sold} with {arg_max} pumps')

The maximum number of gallons sold is 11734.184963026206 with 59 pumps


In [10]:
opt_table

1      3976.040367
2      8473.003651
3     10250.913234
4     10364.723756
5     11113.293859
6     10737.918790
7     10506.802844
8     10481.958043
9     11633.213563
10    10117.413293
11    10720.086500
12    10865.111844
13    11353.273596
14    11347.159480
15    10677.630292
16    10777.197454
17    11131.988820
18    10889.137592
19    10640.754642
20    10589.456550
21    11002.438450
22    10621.821527
23    11109.232444
24    10834.037243
25    11009.509025
26    11234.431900
27    10255.239485
28    10726.317592
29    11245.679572
30    10572.929075
          ...     
70    10956.158761
71    11007.365506
72    10084.331964
73    10913.198752
74    11114.026073
75    10590.365479
76    10112.796772
77    10978.146211
78    10642.773658
79    11186.024628
80    10352.464517
81    10593.969012
82     9971.301690
83    10663.822146
84    11277.014290
85    11025.228510
86    10468.023809
87    11051.512961
88    11175.625228
89    10958.474943
90    10249.676928
91    11039.